In [1]:
!pip install imutils

In [1]:
import cv2
import numpy as np
import imutils
import random
from imutils.perspective import four_point_transform
from imutils import contours


In [13]:
import csv

def create_answer_key(csv_file_path):
    """
    Reads a CSV file and creates a dictionary where the question index starts from 0
    and the answer is the value.

    Args:
        csv_file_path (str): Path to the CSV file.

    Returns:
        dict: Dictionary with question numbers (starting from 0) as keys and answers (as integers) as values.
    """
    answer_key = {}

    # Define a mapping from letters to integers
    letter_to_int = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}  # Assuming each question has 5 options as per the dataset

    with open(csv_file_path, mode='r') as file:
        # Create a CSV reader object
        csv_reader = csv.DictReader(file)

        for row in csv_reader:
            question = int(row['Question']) - 1  # Read 'Question' as integer
            answer_letter = row['Answer'].strip().upper()  # Read 'Answer' as letter, strip whitespace, convert to uppercase
            answer = letter_to_int.get(answer_letter, -1)  # Map letter to integer, -1 if not found

            if answer != -1:
                answer_key[question] = answer  # Assign question as key and answer as value
    
    return answer_key


csv_file_path = 'answer_key.csv'  # Update the path to your CSV file
ANSWER_KEY = create_answer_key(csv_file_path)


In [20]:
# Load the OMR sheet image
image = cv2.imread('omr_2.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

# Find contours of the document
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

if len(cnts) > 0:
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        if len(approx) == 4:
            docCnt = approx
            break

# Get top-down view of the OMR sheet
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

# Apply thresholding to get a binary image
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# Find contours in the thresholded image
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []

# Filter contours to find bubbles
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

# Sort question contours top-to-bottom
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]

correct = 0
questionsContourImage = paper.copy()

# Threshold for considering a bubble as filled
FILL_THRESHOLD = 500  

for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = []
    
    # Loop through each bubble in the question
    for (j, c) in enumerate(cnts):
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)
        
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
        
        if total > FILL_THRESHOLD:  # Consider the bubble as filled
            bubbled.append(j)
    
    color = (0, 0, 255)  # Default: Mark the answer as wrong
    
    # Handle multiple bubbles filled
    if len(bubbled) == 1:
        if ANSWER_KEY[q] == bubbled[0]:
            color = (0, 255, 0)  # Correct answer, mark green
            correct += 1
        else:
            # Draw correct answer in green, wrong one in red
            cv2.drawContours(paper, [cnts[ANSWER_KEY[q]]], -1, (0, 255, 0), 3)
    
    # Multiple bubbles or unfilled
    elif len(bubbled) > 1 or len(bubbled) == 0:
        # Mark bubbles with multiple selections in red
        for idx in bubbled:
            cv2.drawContours(paper, [cnts[idx]], -1, (0, 0, 255), 3)
    
    # Draw the selected bubble in its respective color
    for idx in bubbled:
        cv2.drawContours(paper, [cnts[idx]], -1, color, 3)

# Calculate and display percentage score
score = (correct / len(ANSWER_KEY)) * 100
cv2.putText(paper, f"Score: {score:.2f}%", (10, paper.shape[0] - 20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

# Show the final graded paper
cv2.imshow("Graded OMR Sheet", paper)
cv2.waitKey(0)
cv2.destroyAllWindows()
